In [40]:
%pip install unidecodeç
%pip install chardet
%pip install ftfy


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'unidecodeç'

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
import pandas as pd
import re
import unicodedata
from unidecode import unidecode
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from ftfy import fix_text

- Obtención de datos

In [42]:
ruta_csv = "./resenas_multilingue.csv"
data = []

with open(ruta_csv, encoding="latin1") as f:
    next(f)  # Saltar cabecera
    for line in f:
        parts = line.strip().rsplit(",", 3)
        if len(parts) == 4:
            texto, idioma, rating, categoria = parts
            data.append({
                "texto": texto,
                "idioma": idioma,
                "rating": int(rating),
                "categoria": categoria
            })

df_original = pd.DataFrame(data)
print("📄 Primeras filas del CSV original (sin transformar):")
print(df_original.head())

# Copia para aplicar transformaciones
df = df_original.copy()

📄 Primeras filas del CSV original (sin transformar):
                                               texto idioma  rating  \
0  Ãâ°ste celular es increÃÂ­ble!!! Pero tarda...     es       2   
1  J'adore cet ordinateur! Mais le clavier est tr...     fr       4   
2  This product is a disaster... don't buy it! #W...     en       1   
3  La camisa es bonita, pero se encogiÃÂ³ despuÃ...     es       2   
4  Le service clientÃÂ¨le est nul!!! Je ne recom...     fr       1   

       categoria  
0  electrÃ³nicos  
1  electrÃ³nicos  
2  electrÃ³nicos  
3           ropa  
4      servicios  


- Diccionario de contracciones coloquiales

In [43]:
dic_contracciones = {
    "q'huvo": "que hubo",
    "pq": "porque",
    " x ": " por ",
    "dnd": "donde",
    "tmb": "tambien",
    "xq": "porque",
    "k": "que",
    "q": "que",
    "pa": "para",
    "na": "nada",
    "toi": "estoy",
    "tamos": "estamos",
    "ke": "que",
    "d": "de",
    "la neta": "la verdad",
    "no mms": "no manches",
    "ta": "está",
    "taba": "estaba",
    "io": "yo"
}



- Funciones de normalización

In [44]:
def corregir_codificacion(texto):
    try:
        texto = texto.encode('latin1').decode('utf-8')
    except:
        pass
    texto = fix_text(texto)
    return texto

def normalizar_unicode(texto):
    return unicodedata.normalize('NFC', texto)

def filtrar_simbolos(texto):
    return re.sub(r'[❌▲]', '', texto)

def expandir_contracciones(texto):
    for contra, exp in dic_contracciones.items():
        texto = re.sub(rf'\b{re.escape(contra)}\b', exp, texto, flags=re.IGNORECASE)
    return texto

def eliminar_acentos(texto):
    return unidecode(texto)

def normalizar_hashtags(texto):
    hashtags = re.findall(r'#\w+', texto)
    for h in hashtags:
        h_norm = '#' + unidecode(h[1:]).lower()
        texto = texto.replace(h, h_norm)
    return texto

def pipeline_normalizacion(texto):
    texto = corregir_codificacion(texto)
    texto = normalizar_unicode(texto)
    texto = filtrar_simbolos(texto)
    texto = expandir_contracciones(texto)
    texto = normalizar_hashtags(texto)
    texto = eliminar_acentos(texto)
    texto = texto.lower()
    texto = re.sub(r'[^\w\s#😞😊❤️]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

- Aplicamos pipeline de normalización

In [45]:
df["texto_normalizado"] = df["texto"].apply(pipeline_normalizacion)

-  Mostrar antes/después de algunos casos

In [47]:
print("\n🎯 Ejemplos antes y después de la normalización:")
for i in range(29):
    print("\n---")
    print("Original:", df.loc[i, "texto"])
    print("Normalizado:", df.loc[i, "texto_normalizado"])



🎯 Ejemplos antes y después de la normalización:

---
Original: Ãâ°ste celular es increÃÂ­ble!!! Pero tarda 5hs en cargarse... #BaterÃÂ­aMala ð
Normalizado: este celular es increible pero tarda 5hs en cargarse #bateriamala

---
Original: J'adore cet ordinateur! Mais le clavier est trop petit ð
Normalizado: jadore cet ordinateur mais le clavier est trop petit

---
Original: This product is a disaster... don't buy it! #WasteOfMoney â
Normalizado: this product is a disaster dont buy it #wasteofmoney

---
Original: La camisa es bonita, pero se encogiÃÂ³ despuÃÂ©s del lavado. #CalidadMala
Normalizado: la camisa es bonita pero se encogio despues del lavado #calidadmala

---
Original: Le service clientÃÂ¨le est nul!!! Je ne recommande pas. ð 
Normalizado: le service clientele est nul je ne recommande pas

---
Original: I'm impressed with the battery life!!! Works for 2 days ð
Normalizado: im impressed with the battery life works for 2 days

---
Original: Ãâ¡a marche nick

- Crear etiqueta binaria de sentimiento: 1=negativo (rating ≤ 2), 0=positivo (rating ≥ 4)

In [ ]:
df_modelo = df[df["rating"] != 3].copy()
df_modelo["sentimiento"] = df_modelo["rating"].apply(lambda x: 1 if x <= 2 else 0)

- Vectorizar texto original vs texto normalizado

In [ ]:
vectorizer = CountVectorizer()
X_orig = vectorizer.fit_transform(df_modelo["texto"])
X_norm = vectorizer.fit_transform(df_modelo["texto_normalizado"])
y = df_modelo["sentimiento"]

- División de los datos: Entrenamiento y test.

In [ ]:
X_train_o, X_test_o, y_train, y_test = train_test_split(X_orig, y, test_size=0.2, random_state=42)
X_train_n, X_test_n, _, _ = train_test_split(X_norm, y, test_size=0.2, random_state=42)

- Modelo

In [ ]:
modelo_orig = LogisticRegression(max_iter=300)
modelo_norm = LogisticRegression(max_iter=300)
modelo_orig.fit(X_train_o, y_train)
modelo_norm.fit(X_train_n, y_train)

LogisticRegression(max_iter=300)

- Evaulación del modelo

In [ ]:
acc_orig = accuracy_score(y_test, modelo_orig.predict(X_test_o))
acc_norm = accuracy_score(y_test, modelo_norm.predict(X_test_n))

print("\n📈 COMPARATIVA DE MÉTRICAS")
print("⚖️ Tokens únicos sin normalizar:", len(vectorizer.get_feature_names_out()))
print("⚖️ Accuracy sin normalizar:", round(acc_orig * 100, 2), "%")
print("⚖️ Accuracy con normalización:", round(acc_norm * 100, 2), "%")


📈 COMPARATIVA DE MÉTRICAS
⚖️ Tokens únicos sin normalizar: 155
⚖️ Accuracy sin normalizar: 80.0 %
⚖️ Accuracy con normalización: 80.0 %
